In [668]:
import os
import sys
import pickle

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
stemmer = nltk.PorterStemmer()
from collections import Counter

from bs4 import BeautifulSoup
import re

import warnings
warnings.simplefilter("ignore")

In [463]:
files = os.listdir('news/')
article_types = ['Политика', 'Экономика', 'Происшествия', 'Общество', 'Культура', 'Спорт']
data = pd.DataFrame(columns=['text', 'target'])
articles_none_type = list()

for file in files:
    with open(os.path.join('news', file), 'r') as f:
        raw = f.read()
    soup = BeautifulSoup(raw)
    if len(soup.find_all('item')) >= 3:
        type = soup.find_all('item')[1].string
        if type not in article_types:
            articles_none_type.append(raw)
        else:
            text = soup.find('title').string + '\n' + soup.find('text').string
            data = data.append({'text': text, 'target': type}, ignore_index=True)
    else:
        articles_none_type.append(raw)

In [431]:
print('Датасет')
data

Датасет


,text,target
0,Тувинец сумел отбиться от медведя и откусил ем...,Происшествия
1,Красноярский край станет пилотной площадкой по...,Экономика
2,В тёплые выходные в Красноярске устроят массов...,Происшествия
3,«Боремся с хулиганством и ссорами»: красноярск...,Общество
4,В Бородино стартовал юбилейный сезон трудовых ...,Общество
...,...,...
988,Суд поддержал запрет мэрии на строительство до...,Общество
989,Семья сотрудницы СУЭК стала лучшей в Красноярс...,Общество
990,В Красноярске на глазах у полицейских по пешех...,Происшествия
991,Россиянам могут запретить держать слишком мног...,Общество


In [116]:
print('Статья без определенной категории', BeautifulSoup(articles_none_type[0]), sep='\n\n')

Статья без определенной категории

<?xml version='1.0' encoding='UTF-8'?>
<doc>
<source/><![CDATA[https://newslab.ru/news/902270]]>
<category auto="true" type="str" verify="true"><![CDATA[Новости]]></category>
<title auto="true" type="str" verify="true"><![CDATA[Горожан спросят о развитии красноярских островов]]></title>
<keywords auto="true" type="list" verify="true">
<item type="str"><![CDATA[Новости]]></item>
</keywords>
<text auto="true" type="str" verify="true"><![CDATA[Красноярская администрация на официальном сайте запустила сбор предложений о дальнейшей судьбе трех островов — Отдыха, Молокова и Татышева.
Мэрия планирует создать концепцию развития всех островов. Уже сейчас горожане любят отдыхать, кататься на велосипедах и устраивать пикники на Татышеве, островах Отдыха и Молокова. В мэрии хотят учесть мнение жителей города и предложить удобные и нужные проекты.
В опросе на сайте мэрии красноярцы могут в свободной форме внести свои предложения по развитию территорий. Сбор мнений

In [562]:
data = pd.DataFrame(columns=['text', 'target'])

In [572]:
for file in os.listdir('Dataset/Sport'):
    with open(os.path.join('Dataset/Sport', file), 'r') as f:
        text = f.read()
    data = data.append({'text': text, 'target': 'Спорт'}, ignore_index=True)

In [573]:
data

,text,target
0,Глава Енисейска в Красноярском крае Игорь Анти...,Политика
1,Продолжение: Крайизбирком зарегистрировал трех...,Политика
2,Четырёх депутатов Законодательного Собрания Кр...,Политика
3,Депутаты Горсовета Красноярска потребовали уво...,Политика
4,Губернатор Красноярского края Александр Усс по...,Политика
...,...,...
1495,Российская лыжница Алиса Жамбалова завоевала з...,Спорт
1496,Уже сегодня на главных красноярских аренах отк...,Спорт
1497,Обновлено: Красноярский подросток стал лучшим ...,Спорт
1498,Федерация хоккея России (ФХР) и «Норникель» за...,Спорт


In [574]:
data = data.sample(frac=1).reset_index(drop=True)

In [575]:
class MessageToWordCounterTransform(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = []
        for text in X:
            text = text.lower()
            text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
            text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter([word[:-2] for word in text.split() if len(word) > 3 or word == 'NUMBER'])
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [649]:
class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000, vocabulary_=None):
        self.vocabulary_size = vocabulary_size
        if vocabulary_ is not None:
            with open(vocabulary_, 'rb') as f:
                self.vocabulary_ = pickle.load(f)
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [658]:
pipeline = Pipeline([
    ("email_to_wordcount", MessageToWordCounterTransform()),
    ("wordcount_to_vector", WordCounterToVectorTransformer(vocabulary_='vocabulary.pkl')),
])

In [738]:
X, y = data['text'], data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

X_train = pipeline.transform(X_train).toarray()
X_test = pipeline.transform(X_test).toarray()

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train.values).reshape(-1, 1)
y_test = encoder.transform(y_test.values).reshape(-1, 1)

X = pipeline.transform(X).toarray()
y = encoder.transform(y)

In [739]:
model = SVC(probability=True)

In [740]:
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [741]:
accuracy_score(model.predict(X_test), y_test)

0.6585858585858586

In [750]:
encoder.classes_

array(['Культура', 'Общество', 'Политика', 'Происшествия', 'Спорт',
       'Экономика'], dtype=object)

In [696]:
y_pred = model.predict_proba([X_test[1]])

In [697]:
for pred, type in zip(y_pred[0], article_types):
    print('{} : {}%'.format(type, round(pred * 100, 2)))

Политика : 86.17%
Экономика : 0.99%
Происшествия : 1.87%
Общество : 0.73%
Культура : 7.23%
Спорт : 3.01%


In [703]:
data.iloc[0]['text'].split('.')[0]

'Красноярские депутаты утвердили увеличение бюджета Фонда обязательного медицинского страхования'

In [705]:
data.iloc[0].text

'Красноярские депутаты утвердили увеличение бюджета Фонда обязательного медицинского страхования. Решение об этом было принято 20 декабря на сессии Законодательного Собрания Красноярского края.\nДоходная и расходная часть бюджета территориального фонда обязательного медицинского страхования на 2012 и плановый период 2013-2014 годов увеличены. Сделано это в связи с увеличением трансфертов из бюджета федерального фонда. Так, доходы красноярского фонда увеличиваются на 1 млрд 235,9 млн рублей, расходы возрастают на 1 млрд 185,9 млн рублей, источники внутреннего финансирования дефицита бюджета фонда уменьшаются на 50 млн рублей.\nПолученные дополнительно средства будут потрачены на следующие направления: модернизация здравоохранения Красноярского края в части укрепления материально-технической базы медицинских учреждений, выполнение переданных органам государственной власти края полномочий РФ в сфере обязательного медицинского страхования, реализация национального проекта «Здоровье» в част

In [727]:
text = 'Красноярские депутаты утвердили увеличение бюджета Фонда обязательного медицинского страхования. Решение об этом было принято 20 декабря на сессии Законодательного Собрания Красноярского края.\nДоходная и расходная часть бюджета территориального фонда обязательного медицинского страхования на 2012 и плановый период 2013-2014 годов увеличены. Сделано это в связи с увеличением трансфертов из бюджета федерального фонда. Так, доходы красноярского фонда увеличиваются на 1 млрд 235,9 млн рублей, расходы возрастают на 1 млрд 185,9 млн рублей, источники внутреннего финансирования дефицита бюджета фонда уменьшаются на 50 млн рублей.\nПолученные дополнительно средства будут потрачены на следующие направления: модернизация здравоохранения Красноярского края в части укрепления материально-технической базы медицинских учреждений, выполнение переданных органам государственной власти края полномочий РФ в сфере обязательного медицинского страхования, реализация национального проекта «Здоровье» в части проведения диспансеризации и осуществление единовременных компенсационных выплат медицинским работникам.\nЗа год на обязательное медстрахование в Красноярском крае планируют потратить 32,5 млрд рублей'

In [728]:
vector = pipeline.transform([text])

In [737]:
model.predict_proba(vector.toarray())[0].tolist()

[0.027195104392764086,
 0.3115741411227261,
 0.40705833916508494,
 0.03615629052747016,
 0.0544613579727595,
 0.16355476681919556]

In [730]:
vector.toarray().shape

(1, 1001)

In [723]:
with open('pipeline.pickle', 'wb') as handle:
    pickle.dump(pipeline, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [726]:
with open('pipeline.pickle', 'rb') as handle:
    pipeline = pickle.load(handle)